In [1]:
import os
import string
import math

In [2]:
X_train = [] # an element of X is represented as (filename,text)
Y_train = [] # an element of Y represents the newsgroup category of the corresponding X element
count=0
for category in os.listdir('20news-bydate/20news-bydate-train'):
    for document in os.listdir('20news-bydate/20news-bydate-train/'+category):
        with open('20news-bydate/20news-bydate-train/'+category+'/'+document, "r") as f:
            try:
                X_train.append((document,f.read()))
                Y_train.append(category)
            except:
                count+=1

In [3]:
X_test = [] # an element of X is represented as (filename,text)
Y_test = [] # an element of Y represents the newsgroup category of the corresponding X element
testcount=0
for category in os.listdir('20news-bydate/20news-bydate-test'):
    for document in os.listdir('20news-bydate/20news-bydate-test/'+category):
        with open('20news-bydate/20news-bydate-test/'+category+'/'+document, "r") as f:
            try:
                X_test.append((document,f.read()))
                Y_test.append(category)
            except:
                testcount+=1

In [4]:
#stopwords
f = open('en.stopword.txt','r',encoding="utf-8")
stopwords = f.read().split("\n")

In [5]:
#vocab is unique words, term frequency
vocab = {}
for i in range(len(X_train)):
    word_list = []
    for word in X_train[i][1].split():
        word_new  = word.strip(string.punctuation).lower()
        if (len(word_new)>2)  and (word_new not in stopwords):  
            if word_new in vocab:
                vocab[word_new]+=1
            else:
                vocab[word_new]=1 

In [6]:
#no of documents is certain class(Nc)
docclasscount = {}
for x in Y_train:
    if x in docclasscount:
        docclasscount[x]+=1
    else:
        docclasscount[x]=1

In [7]:
N = len(Y_train)

In [8]:
#prior probability of the each class
prior = {}
for x in docclasscount:
    prior[x] = docclasscount[x]/N

In [9]:
#textc has term frequency of each word in each class
textc = {}
classcount = {}
for word in vocab:
    textc[word] = {}
    for x in set(Y_train):
        textc[word][x] = 0
        classcount[x] = 0

In [10]:
#textc calculation
for i in range(len(X_train)):
    currclass = Y_train[i]
    for word in X_train[i][1].split():
        word_new  = word.strip(string.punctuation).lower()
        if (len(word_new)>2)  and (word_new not in stopwords):
            textc[word_new][currclass]+=1

In [11]:
#classcount has count of all words in certain class documents 
for word in vocab:
    for x in set(Y_train):
        classcount[x] += textc[word][x]

In [12]:
# conditinal probalilty calculation
condprob = {}
B = len(vocab)
for word in vocab:
    condprob[word] = {}
    for x in set(Y_train):
        condprob[word][x] = (textc[word][x]+1)/(classcount[x] + B)

In [13]:
#Test dataset 
Y_pred=[]
for i in range(len(X_test)):
    word_list = []
    for word in X_test[i][1].split():
        word_new  = word.strip(string.punctuation).lower()
        if (len(word_new)>2)  and (word_new not in stopwords):
            word_list.append(word_new)
            
    score = {}
    for x in set(Y_train):
        score[x] = math.log(prior[x])
    
    for x in set(Y_train):
        for word in word_list:
            try:
                score[x] += math.log(condprob[word][x])
            except:
                continue
            
    max_key = max(score, key=score.get)
    Y_pred.append(max_key)
            

In [14]:
count = 0
for i in range(len(Y_pred)):
    if Y_pred[i] == Y_test[i]:
        count+=1
accuracy = count/len(Y_pred)

In [15]:
print("Our score on testing data :",accuracy)

Our score on testing data : 0.8150073304011729
